In [1]:
import pandas as pd
import numpy as np

import pycountry

import altair as alt

In [2]:
%%html
<style>
@import url('https://fonts.googleapis.com/css2?family=IBM+Plex+Sans:wght@500&display=swap');
</style>

### DATA

In [3]:
fsi = pd.read_csv('data/fsi-2022.csv')

In [4]:
bp2021 = pd.read_csv('data/bp-stats-review-2021-consolidated-dataset-narrow-format.csv')

In [5]:
bp2021.rename(columns={
    'ISO3166_alpha3': 'alpha3'
}, inplace=True)

bp2021 = bp2021.loc[ ~bp2021.alpha3.isna() & (bp2021.Year == 2020)]

### WORLD TOTALS

In [6]:
world = pd.read_csv('data/bp-stats-review-2021-consolidated-dataset-narrow-format.csv')

world = world.loc[
    (world.Var.isin(['oilreserves_bbl', 'gasreserves_tcm'])) & (world.Year == 2020) & (world.Country == 'Total World')
]

In [7]:
world.loc[world.Var == 'oilreserves_bbl', 'Value'].iloc[0]

1732.366175

In [8]:
world.loc[world.Var == 'gasreserves_tcm', 'Value'].iloc[0]

188.0742261

### COUNTRIES AND MERGE

In [9]:
def get_alpha3(country):
    try:
        alpha3 = pycountry.countries.search_fuzzy(country)[0].alpha_3
    except LookupError as e:
        alpha3 = None
        
    return alpha3
        

fsi.loc[:, 'alpha3'] = fsi.loc[:, 'Country'].apply(
    get_alpha3
)

In [10]:
alpha3_codes = {
    'Congo Democratic Republic': 'COD',
    'Congo Republic': 'COG',
    'Guinea Bissau': 'GNB',
    'North Korea': 'PRK',
    'Laos': 'LAO',
    'South Korea': 'KOR',
    'Nigeria': 'NGA',
    'Niger': 'NER'
}

for country, code in alpha3_codes.items():
    fsi.loc[fsi.Country == country, 'alpha3'] = code

In [11]:
data = bp2021.merge(
    fsi.loc[:, ['Total', 'Rank', 'alpha3']],
    on='alpha3',
    how='left'
)

In [12]:
data = data.loc[
    ~data.Total.isna(),
    ['Country', 'Region', 'SubRegion', 'Var', 'Value', 'Total', 'Rank']
].rename(columns={
    'Total' : 'FSI', 'Rank': 'FSIRank'
})

In [13]:
data.head()

,Country,Region,SubRegion,Var,Value,FSI,FSIRank
0,Algeria,Africa,Northern Africa,wind_twh,0.008000,72.2,77th
1,Argentina,S. & Cent. America,South America,wind_twh,9.470298,47.9,138th
2,Australia,Asia Pacific,Asia Pacific,wind_twh,22.606892,22.7,168th
3,Austria,Europe,Europe,wind_twh,6.794140,25.4,166th
4,Azerbaijan,CIS,CIS,wind_twh,0.095100,73.1,73rd


###

### OIL RESERVES AND FRAGILITY

In [14]:
oil_reserves = data.loc[data.Var == 'oilreserves_bbl']
oil_reserves.loc[:, 'Share'] = oil_reserves.loc[:, 'Value'] / world.loc[world.Var == 'oilreserves_bbl', 'Value'].iloc[0]

/home/d2ski/miniconda3/envs/vizlab/lib/python3.7/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [15]:
def fsi_levels(fsi):
    if fsi < 30:
        return 'Very sustainable'
    elif fsi < 40:
        return 'Sustainable'
    
    elif fsi < 50:
        return 'More stable'
    elif fsi < 60:
        return 'Stable'
    elif fsi < 70:
        return 'Less stable'
    
    elif fsi < 80:
        return 'Warning'
    elif fsi < 90:
        return 'Elevated warning'
    elif fsi < 100:
        return 'High warning'
    
    elif fsi < 110:
        return 'Alert'
    elif fsi < 120:
        return 'High alert'
    else:
        return 'Very high alert'
    
    
def get_fsi_label(fsi):
    if fsi < 40:
        return 'SUSTAINABLE'
    if fsi < 70:
        return 'STABLE'
    if fsi < 100:
        return 'WARNING'
    
    return 'ALERT'
    
oil_reserves.loc[
    :, 'FSILabel'
] = oil_reserves.FSI.apply(get_fsi_label)

oil_reserves.loc[
    :, 'FSILevel'
] = oil_reserves.FSI.apply(fsi_levels)

In [16]:
oil_reserves.sort_values('Share', ascending=False).head(10)

,Country,Region,SubRegion,Var,Value,FSI,FSIRank,Share,FSILabel,FSILevel
1304,Venezuela,S. & Cent. America,South America,oilreserves_bbl,303.805745,91.6,26th,0.175370,WARNING,High warning
1292,Saudi Arabia,Middle East,Middle East,oilreserves_bbl,297.527000,67.5,95th,0.171746,STABLE,Less stable
1265,Canada,North America,North America,oilreserves_bbl,168.087611,20.1,172nd,0.097028,SUSTAINABLE,Very sustainable
1276,Iran,Middle East,Middle East,oilreserves_bbl,157.800000,84.1,39th,0.091089,WARNING,Elevated warning
1277,Iraq,Middle East,Middle East,oilreserves_bbl,145.019000,93.8,23rd,0.083712,WARNING,High warning
1291,Russian Federation,CIS,CIS,oilreserves_bbl,107.804239,72.6,75th,0.062229,WARNING,Warning
1280,Kuwait,Middle East,Middle East,oilreserves_bbl,101.500000,52.2,130th,0.058590,STABLE,Stable
1301,United Arab Emirates,Middle East,Middle East,oilreserves_bbl,97.800000,39.1,152nd,0.056455,SUSTAINABLE,Sustainable
1300,US,North America,North America,oilreserves_bbl,68.757000,46.6,140th,0.039690,STABLE,More stable
1281,Libya,Africa,Northern Africa,oilreserves_bbl,48.363000,94.3,21st,0.027917,WARNING,High warning


In [17]:
oil_reserves.groupby('FSILabel').agg({
    'Share': 'sum',
    'Country': 'count'
})

,Share,Country
FSILabel,,
ALERT,0.006927,5
STABLE,0.333147,22
SUSTAINABLE,0.159670,5
WARNING,0.495632,17


In [18]:
colors = list(reversed(['#67001f','#b2182b','#d6604d','#f4a582','#fddbc7','#ffffff','#e0e0e0','#bababa','#878787','#4d4d4d','#1a1a1a']))
levels = [
    'Very sustainable',
    'Sustainable',
    'More stable',
    'Stable',
    'Less stable',
    'Warning',
    'Elevated warning',
    'High warning',
    'Alert',
    'High alert',
    'Very high alert',
]

background_color = '#00796B'
stroke_color = '#fff'

In [19]:
y_ticks = ['ALERT', 'WARNING', 'STABLE', 'SUSTAINABLE']
TICKS_FONT_SIZE = 14
TICKS_FONT_WEIGHT = 500
TICKS_FONT = 'IBM Plex Sans'
TICKS_SIZE = 6
TICKS_WIDTH = 2
TICKS_COLOR = '#fafafa'

oil_chart = alt.Chart(
    oil_reserves
).mark_bar(strokeWidth=0.5, stroke=background_color).encode(
    x=alt.X(
        'sum(Share)',
        axis=alt.Axis(
            grid=False,
            domain=False,
            tickCount=4,
            labelExpr="datum.value*100 +'%'",
            labelFont=TICKS_FONT,
            labelColor=stroke_color,
            labelFontSize=TICKS_FONT_SIZE,
            labelFontWeight=TICKS_FONT_WEIGHT,
            labelPadding=8,
            labelAlign='center',
            title='',
            tickSize=TICKS_SIZE,
            tickWidth=TICKS_WIDTH,
            tickColor=TICKS_COLOR,
            offset=10,
        )
    ),
    y=alt.Y(
        'FSILabel:O',
        sort=y_ticks,
        axis=alt.Axis(
            grid=False,
            domain=False,
            labelFont=TICKS_FONT,
            labelColor=stroke_color,
            labelFontSize=16,
            labelFontWeight=TICKS_FONT_WEIGHT,
            labelPadding=16,
            title='',
            tickSize=0,
            tickWidth=TICKS_WIDTH,
            tickColor=TICKS_COLOR,
        )),
    color=alt.Color('FSILevel', scale=alt.Scale(
        domain=levels, range=colors
    )),
    detail='Country',
    order=alt.Order('Share'),
    tooltip=['Country', 'Share', 'FSILevel']
).properties(
    width=1000,
    height=400
)

### NATURAL GAS RESERVES AND FRAGILITY

In [20]:
gas_reserves = data.loc[data.Var == 'gasreserves_tcm']
gas_reserves.loc[:, 'Share'] = gas_reserves.loc[:, 'Value'] / world.loc[world.Var == 'gasreserves_tcm', 'Value'].iloc[0]

gas_reserves.loc[
    :, 'FSILabel'
] = gas_reserves.FSI.apply(get_fsi_label)

gas_reserves.loc[
    :, 'FSILevel'
] = gas_reserves.FSI.apply(fsi_levels)

/home/d2ski/miniconda3/envs/vizlab/lib/python3.7/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [21]:
gas_reserves.sort_values('Share', ascending=False).head(10)

,Country,Region,SubRegion,Var,Value,FSI,FSIRank,Share,FSILabel,FSILevel
2419,Russian Federation,CIS,CIS,gasreserves_tcm,37.391524,72.6,75th,0.198813,WARNING,Warning
2399,Iran,Middle East,Middle East,gasreserves_tcm,32.101379,84.1,39th,0.170685,WARNING,Elevated warning
2417,Qatar,Middle East,Middle East,gasreserves_tcm,24.665471,42.3,147th,0.131148,STABLE,More stable
2424,Turkmenistan,CIS,CIS,gasreserves_tcm,13.601250,66.5,102nd,0.072319,STABLE,Less stable
2425,US,North America,North America,gasreserves_tcm,12.618704,46.6,140th,0.067094,STABLE,More stable
2392,China,Asia Pacific,Asia Pacific,gasreserves_tcm,8.398550,66.9,98th,0.044656,STABLE,Less stable
2430,Venezuela,S. & Cent. America,South America,gasreserves_tcm,6.260173,91.6,26th,0.033286,WARNING,High warning
2420,Saudi Arabia,Middle East,Middle East,gasreserves_tcm,6.019120,67.5,95th,0.032004,STABLE,Less stable
2427,United Arab Emirates,Middle East,Middle East,gasreserves_tcm,5.938725,39.1,152nd,0.031576,SUSTAINABLE,Sustainable
2410,Nigeria,Africa,Western Africa,gasreserves_tcm,5.472979,97.2,16th,0.029100,WARNING,High warning


In [22]:
gas_reserves.groupby('FSILabel').agg({
    'Share': 'sum',
    'Country': 'count'
})

,Share,Country
FSILabel,,
ALERT,0.005140,3
STABLE,0.408655,23
SUSTAINABLE,0.065342,7
WARNING,0.510714,18


In [23]:
y_ticks = ['ALERT', 'WARNING', 'STABLE', 'SUSTAINABLE']
TICKS_FONT_SIZE = 14
TICKS_FONT_WEIGHT = 500
TICKS_FONT = 'IBM Plex Sans'
TICKS_SIZE = 6
TICKS_WIDTH = 2
TICKS_COLOR = '#fafafa'

gas_chart = alt.Chart(
    gas_reserves
).mark_bar(strokeWidth=0.5, stroke=background_color).encode(
    x=alt.X(
        'sum(Share)',
        axis=alt.Axis(
            grid=False,
            domain=False,
            tickCount=4,
            labelExpr="datum.value*100 +'%'",
            labelFont=TICKS_FONT,
            labelColor=stroke_color,
            labelFontSize=TICKS_FONT_SIZE,
            labelFontWeight=TICKS_FONT_WEIGHT,
            labelPadding=8,
            labelAlign='center',
            title='',
            tickSize=TICKS_SIZE,
            tickWidth=TICKS_WIDTH,
            tickColor=TICKS_COLOR,
            offset=10,
        )
    ),
    y=alt.Y(
        'FSILabel:O',
        sort=y_ticks,
        axis=alt.Axis(
            grid=False,
            domain=False,
            labelFont=TICKS_FONT,
            labelColor=stroke_color,
            labelFontSize=16,
            labelFontWeight=TICKS_FONT_WEIGHT,
            labelPadding=16,
            title='',
            tickSize=0,
            tickWidth=TICKS_WIDTH,
            tickColor=TICKS_COLOR,
        )),
    color=alt.Color('FSILevel', scale=alt.Scale(
        domain=levels, range=colors
    )),
    detail='Country',
    order=alt.Order('Share'),
    tooltip=['Country', 'Share', 'FSILevel']
).properties(
    width=1000,
    height=400
)

In [24]:
alt.vconcat(oil_chart, gas_chart).configure(
    background=background_color
).configure_view(
    strokeWidth=0,
).resolve_scale(x='shared')

alt.VConcatChart(...)

### SAVE DATASET

In [26]:
pd.concat(
    [oil_reserves, gas_reserves],
    ignore_index=True
).to_csv('data/dataset.csv')